In [ ]:
using System.IO;

var filePath = @"D:\test\Chrome.lnk";

var fi = new FileInfo(filePath);

var writeTime = fi.LastWriteTimeUtc.Ticks;
var length = fi.Length;
var fa = fi.Attributes;
var fileAttributes = (int)fa;
//so far we are doing what client tools doing

bool isSymbolicLink = (fa & FileAttributes.ReparsePoint) == FileAttributes.ReparsePoint;
isSymbolicLink

In [ ]:
using System.IO;
using System.ComponentModel;
using System.Runtime.InteropServices;

readonly IntPtr INVALID_HANDLE_VALUE = new IntPtr(-1);

const uint FILE_READ_EA = 0x0008;
const uint FILE_FLAG_BACKUP_SEMANTICS = 0x2000000;

[DllImport("Kernel32.dll", SetLastError = true, CharSet = CharSet.Auto)]
static extern uint GetFinalPathNameByHandle(IntPtr hFile, [MarshalAs(UnmanagedType.LPTStr)] StringBuilder lpszFilePath, uint cchFilePath, uint dwFlags);

[DllImport("kernel32.dll", SetLastError = true)]
[return: MarshalAs(UnmanagedType.Bool)]
static extern bool CloseHandle(IntPtr hObject);

[DllImport("kernel32.dll", CharSet = CharSet.Auto, SetLastError = true)]
static extern IntPtr CreateFile(
        [MarshalAs(UnmanagedType.LPTStr)] string filename,
        [MarshalAs(UnmanagedType.U4)] uint access,
        [MarshalAs(UnmanagedType.U4)] FileShare share,
        IntPtr securityAttributes, // optional SECURITY_ATTRIBUTES struct or IntPtr.Zero
        [MarshalAs(UnmanagedType.U4)] FileMode creationDisposition,
        [MarshalAs(UnmanagedType.U4)] uint flagsAndAttributes,
        IntPtr templateFile);

string GetFinalPathName(string path)
{
    var h = CreateFile(path,
        FILE_READ_EA,
        FileShare.ReadWrite | FileShare.Delete,
        IntPtr.Zero,
        FileMode.Open,
        FILE_FLAG_BACKUP_SEMANTICS,
        IntPtr.Zero);
    if (h == INVALID_HANDLE_VALUE)
        throw new Win32Exception();

    try
    {
        var sb = new StringBuilder(1024);
        var res = GetFinalPathNameByHandle(h, sb, 1024, 0);
        if (res == 0)
            throw new Win32Exception();

        var finalPath = sb.ToString();
        if (finalPath.StartsWith(@"\\?\"))
            return finalPath.Substring(4);
        else
            return finalPath;
    }
    finally
    {
        CloseHandle(h);
    }
}

In [ ]:
var symbolicLinkPath = @"D:\test\Chrome.lnk";
GetFinalPathName(symbolicLinkPath)

In [ ]:
var symbolicLinkPath = @"D:\test\src";
GetFinalPathName(symbolicLinkPath)

In [ ]:
using System;
using System.IO;
using System.Runtime.InteropServices;




const int SYMBOLIC_LINK_FLAG_DIRECTORY = 0x1;
const int SYMBOLIC_LINK_FLAG_ALLOW_UNPRIVILEGED_CREATE = 0x2;
const int MaxShortPath = 260;

[LibraryImport("kernel32.dll", EntryPoint = "CreateSymbolicLinkW", SetLastError = true, StringMarshalling = StringMarshalling.Utf16)]
[return: MarshalAs(UnmanagedType.U1)]
partial bool CreateSymbolicLinkPrivate(string lpSymlinkFileName, string lpTargetFileName, int dwFlags);


static bool IsPartiallyQualified(ReadOnlySpan<char> path)
{
    if (path.Length < 2)
        return true;

    if (IsDirectorySeparator(path[0]))
    {
        return !(path[1] == '?' || IsDirectorySeparator(path[1]));
    }

    return !((path.Length >= 3)
        && (path[1] == VolumeSeparatorChar)
        && IsDirectorySeparator(path[2])
        && IsValidDriveChar(path[0]));
}


static string EnsureExtendedPrefix(string path)
{
    if (IsPartiallyQualified(path.AsSpan()) || IsDevice(path.AsSpan()))
        return path;

    // Given \\server\share in longpath becomes \\?\UNC\server\share
    if (path.StartsWith(UncPathPrefix, StringComparison.OrdinalIgnoreCase))
        return path.Insert(2, UncExtendedPrefixToInsert);

    return ExtendedPathPrefix + path;
}

static string EnsureExtendedPrefixIfNeeded(string path)
{
    if (path != null && (path.Length >= MaxShortPath || EndsWithPeriodOrSpace(path)))
    {
        return EnsureExtendedPrefix(path);
    }
    else
    {
        return path;
    }
}



void CreateSymbolicLink(string symlinkFileName, string targetFileName, bool isDirectory)
{
    string originalPath = symlinkFileName;
    symlinkFileName = PathInternal.EnsureExtendedPrefixIfNeeded(symlinkFileName);
    targetFileName = PathInternal.EnsureExtendedPrefixIfNeeded(targetFileName);

    int flags = 0;

    Version osVersion = Environment.OSVersion.Version;
    bool isAtLeastWin10Build14972 = osVersion.Major >= 11 || osVersion.Major == 10 && osVersion.Build >= 14972;

    if (isAtLeastWin10Build14972)
    {
        flags = SYMBOLIC_LINK_FLAG_ALLOW_UNPRIVILEGED_CREATE;
    }

    if (isDirectory)
    {
        flags |= SYMBOLIC_LINK_FLAG_DIRECTORY;
    }

    bool success = CreateSymbolicLinkPrivate(symlinkFileName, targetFileName, flags);

    if (!success)
    {
        throw Win32Marshal.GetExceptionForLastWin32Error(originalPath);
    }
}

In [ ]:
using System.Runtime.InteropServices;

[Flags]
enum SYMBOLIC_LINK_FLAG
{
    File = 0,
    Directory = 1,
    AllowUnprivilegedCreate = 2
}

const int SYMBOLIC_LINK_FLAG_ALLOW_UNPRIVILEGED_CREATE = 0x2;
const int SYMBOLIC_LINK_FLAG_DIRECTORY = 0x1;

[DllImport("kernel32.dll", EntryPoint = "CreateSymbolicLinkW", CharSet = CharSet.Unicode, SetLastError = true)]
static extern int CreateSymbolicLink(string lpSymlinkFileName, string lpTargetFileName, int dwFlags);


bool isDirectory = false;
var target = @"C:\Program Files\Google\Chrome\Application\chrome.exe";

Version osVersion = Environment.OSVersion.Version;
bool isAtLeastWin10Build14972 = osVersion.Major >= 11 || osVersion.Major == 10 && osVersion.Build >= 14972;

int flags = 0;
if (isAtLeastWin10Build14972) flags = SYMBOLIC_LINK_FLAG_ALLOW_UNPRIVILEGED_CREATE;
if (isDirectory) flags |= SYMBOLIC_LINK_FLAG_DIRECTORY;

Console.WriteLine($"Using flags {flags}");

var result = CreateSymbolicLink(@"D:\test\chrome.lnk", target, flags);
if (result == 1)
    Console.WriteLine("Created");
else
{
    int w32Error = Marshal.GetLastWin32Error(); //we can return 0; this will always be some positive value
    Console.WriteLine($"Failed with {w32Error}");
}